In [2]:
import pandas as pd
import os
from core import read_mmseqs_results

In [3]:
model_seq_info = pd.read_parquet('../data3/interim/model_seq_info.pq')

In [4]:
model_seq_f = '../data3/interim/model_seqs.faa'

In [4]:
mmseqs_search_out = '../data3/interim/model_seq_search.txt'

In [5]:
%%time
os.system(' '.join(['conda run -n beaker', 
                    'mmseqs easy-search', 
                    model_seq_f, 
                    model_seq_f, 
                    mmseqs_search_out,
                    '../data/tmp',
                    '--format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov', 
                    '--threads 48',
                    '-s 7.5']))

../data3/interim/model_seq_search.txt exists and will be overwritten
easy-search ../data3/interim/model_seqs.faa ../data3/interim/model_seqs.faa ../data3/interim/model_seq_search.txt ../data/tmp --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov --threads 48 -s 7.5 

MMseqs Version:                        	13.45111
Substitution matrix                    	nucl:nucleotide.out,aa:blosum62.out
Add backtrace                          	false
Alignment mode                         	3
Alignment mode                         	0
Allow wrapped scoring                  	false
E-value threshold                      	0.001
Seq. id. threshold                     	0
Min alignment length                   	0
Seq. id. mode                          	0
Alternative alignments                 	0
Coverage threshold                     	0
Coverage mode                          	0
Max sequence length                    	65535
Compositional bias              

0

In [5]:
%%time
mmseqs_search_df = read_mmseqs_results(mmseqs_search_out)

CPU times: user 26 s, sys: 3.43 s, total: 29.4 s
Wall time: 29.9 s


**Query:** test gene

**Target:** train gene

In [6]:
top_train_list = list()
for fold, fold_test_df in model_seq_info.groupby('test_fold'):
    print(fold)
    top_mmseqs_df = (mmseqs_search_df[mmseqs_search_df['query'].isin(fold_test_df['seq_id']) & 
                                      ~mmseqs_search_df['target'].isin(fold_test_df['seq_id'])]
                     .sort_values('bits', ascending=False)
                     .groupby('query')
                     .head(1))
    top_train_list.append(top_mmseqs_df)

0
1
2
3
4


In [7]:
cat_top_hit = pd.concat(top_train_list)

In [8]:
leaked_data = cat_top_hit[((cat_top_hit['qcov'] > 0.8) | 
                           (cat_top_hit['tcov'] > 0.8)) & 
                          (cat_top_hit['fident'] > 0.3)]

In [9]:
merged_leaked_data = (leaked_data.merge(model_seq_info
                                         .rename(columns={'seq_id': 'query'}), 
                                         how='inner', on='query')
                      .merge(model_seq_info
                             .rename(columns={'seq_id': 'target'}), 
                             how='inner', on='target', suffixes=('_query', '_target')))

In [10]:
pd.set_option('display.max_colwidth', 100)
merged_leaked_data[['query', 'target', 'qcov', 'tcov', 'fident', 'defensive_query', 'defensive_target']]

,query,target,qcov,tcov,fident,defensive_query,defensive_target
0,58716cd0f8d8332725ba61f467c8b195330aeb9267176c9a589765db,27170489b598945be8f2540c4e3d172e6f206c0411643fcb1b7c0eda,0.999,0.999,0.422,False,False
1,8d05d5458982cbd3c21f7a0f4b357bfd954940adb13cd5f8a9c2f984,27170489b598945be8f2540c4e3d172e6f206c0411643fcb1b7c0eda,0.990,0.887,0.314,False,False
2,774022f6bbc615deab88403dfdaad8ee2ff1a6fbf8cf2da1223cf0ab,27170489b598945be8f2540c4e3d172e6f206c0411643fcb1b7c0eda,0.981,0.698,0.305,False,False
3,70c88d578a5af1e89f83c464c7ebc691f638961eef107fb4ea94da6b,27170489b598945be8f2540c4e3d172e6f206c0411643fcb1b7c0eda,0.805,0.047,0.315,False,False
4,49cb910efe0fa88b16eb2150960766bf90c3ed91298288b8ececf37d,90722b86e0e71131620cec19bdacc39f959e6ed61186d61420f8cedf,0.703,0.910,0.718,False,False
...,...,...,...,...,...,...,...
6005,da598f7cb0033405abd0ce2fa6c5aed08da87e14e2b93085d364348d,aa1a9f256f542b11ffa90a5d1a81fc37a80e7de6d07acb6d90a33b12,0.913,0.223,0.476,False,False
6006,10e33d97fd5d18dccff06414a4a8e8a06e89b3c83feb1bb526fd0adf,e06493b884afe136b6ec5a4c793d02446ef4b306d48e46a5f60bc398,0.889,0.851,0.375,False,False
6007,7a4f38a9fcf54a1da17c4cf3c680695db9494d1898e08832cc6fec25,9492fdd2586010e0d7977dd4964a8e8cf581c6307afd4c508990a9e5,0.885,0.730,0.456,False,False
6008,14ae9af5f3720af41ff4f7fa7784c9e5ece8e2031c0bb91dffde1478,13007df7b3eff484077bf3403ad3daa83642225bdbb9976e696fd8b0,0.952,0.231,0.348,False,False


In [11]:
merged_cat_top_hit = (cat_top_hit[['query', 'target', 'bits']]
                      .merge(model_seq_info[['seq_id', 'defensive']]
                             .rename(columns={'seq_id': 'target', 
                                              'defensive': 'target_defensive'}), 
                             how='inner', on='target'))
merged_cat_top_hit['direction'] = merged_cat_top_hit['target_defensive'] * 2 - 1
merged_cat_top_hit['prediction'] = merged_cat_top_hit['bits'] * merged_cat_top_hit['direction']

In [12]:
out_df = (model_seq_info[['seq_id']]
          .merge(merged_cat_top_hit[['query', 'prediction']]
                 .rename(columns={'query': 'seq_id'}), 
                 how='left', on='seq_id'))
out_df['prediction'] = out_df['prediction'].fillna(0)
out_df['method'] = 'MMseqs nearest neighbor'

In [13]:
out_df.to_parquet('../data3/interim/cv_predictions_mmseqs.pq', index=False)